In [ ]:
import scanpy as sc
import besca as bc
import os
import bbknn
import numpy as np
import scvelo as scv
import pandas as pd

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss
from scipy import stats
import itertools


In [ ]:
color_dict = {'R': 'coral', 'TF': 'firebrick', 'R_nadj': 'lightskyblue','R_adj': 'royalblue'}


In [ ]:
sc.logging.print_versions()

# Reading the h5ad

In [ ]:
WD=''
analysis_name='sw_besca2_immune'
#dcdata=sc.read(WD+'analyzed/'+analysis_name+'/'+analysis_name+'.annotated.h5ad')
dcdata=sc.read('/Fullanalysis/analyzed/sw_besca2_immune/sw_besca2_immune.annotated.h5ad')

In [ ]:
outpath='/Fullanalysis/analyzed/sw_besca2_immune/velocity/'

In [ ]:
velosubset='MonoMacro'

In [ ]:
set(dcdata.obs['celltype2_merged'])

In [ ]:
sc.set_figure_params()

In [ ]:
sc.pl.umap(dcdata, color=['leiden', 'leiden_TNK', 'Sample type'],legend_loc='on data', legend_fontsize=0.6)

In [ ]:
sc.set_figure_params()
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,10)

In [ ]:
sc.pl.umap(dcdata, color=['celltype3_rc_cells'], size=8)

In [ ]:
sc.pl.umap(dcdata, color=['celltype3_merged'], size=8)

In [ ]:
sc.set_figure_params()
sc.pl.umap(dcdata,color=['CD8A','CD8B','CD4','CD3D'], color_map='viridis')

In [ ]:
set(dcdata.obs['celltype2_merged'])

In [ ]:
#dcdata=dcdata[(dcdata.obs['celltype2'].isin(['CD8-positive, alpha-beta T cell']))|(dcdata.obs['celltype2_rc_cells'].isin(['CD8-positive, alpha-beta T cell']))|(dcdata.obs['celltype2_rc_sample'].isin(['CD8-positive, alpha-beta T cell']))].copy()
#dcdata=dcdata[(dcdata.obs['celltype2'].isin(['CD8-positive, alpha-beta T cell']))|(dcdata.obs['celltype2_rc_cells'].isin(['CD8-positive, alpha-beta T cell']))].copy()
dcdata=dcdata[(dcdata.obs['celltype2_merged'].isin(['macrophage', 'classical monocyte',
                                                    'non-classical monocyte','myeloid leukocyte']))].copy()
#dcdata=dcdata[(dcdata.obs['celltype3_merged']!='proliferating CD4-positive, alpha-beta T cell')].copy()

In [ ]:
#dcdata=dcdata[(dcdata.obs['celltype3_rc_cells']!='proliferating CD4-positive, alpha-beta T cell')].copy()
#dcdata=dcdata[(dcdata.obs['celltype3_rc_sample']!='proliferating CD4-positive, alpha-beta T cell')].copy()
#dcdata=dcdata[(dcdata.obs['celltype3']!='naive thymus-derived CD4-positive, alpha-beta T cell')].copy()
#dcdata=dcdata[(dcdata.obs['celltype3_rc_cells']!='naive thymus-derived CD4-positive, alpha-beta T cell')].copy()
#dcdata=dcdata[(dcdata.obs['celltype3']!='central memory CD4-positive, alpha-beta T cell')].copy()
#dcdata=dcdata[(dcdata.obs['celltype3']!='CD4-positive, alpha-beta cytotoxic T cell')].copy()
#dcdata=dcdata[(dcdata.obs['celltype1_rc_cells']=='T cell')].copy()


#dcdata=dcdata[dcdata.obs['celltype2_TNK'].isin(['CD8Tcell'])]
#dcdata=dcdata[~dcdata.obs['leiden_TNK'].isin(['14','34','3','31'])].copy()

In [ ]:
#sub=(((dcdata.raw[:,'CD8A'].X.todense()+dcdata.raw[:,'CD8B'].X.todense())>0)|(dcdata.raw[:,'CD4'].X.todense()==0))

In [ ]:
dcdata.obs['leidenf']=dcdata.obs['leiden'].copy()

In [ ]:
sc.pl.umap(dcdata, color=['leiden', 'celltype3_merged','celltype3', 'Sample type'])

In [ ]:
sc.pl.umap(dcdata, color=['celltype3_merged'],legend_fontsize=8) #14, 34, 3, 31


In [ ]:
sc.pl.umap(dcdata, color=['celltype3'],legend_fontsize=8) #14, 34, 3, 31


In [ ]:
sc.pl.umap(dcdata, color=['CLEC9A','CLEC10A','MSR1'])

In [ ]:
sc.pl.umap(dcdata, color=['celltype3_rc_cells'],legend_fontsize=8) #14, 34, 3, 31


In [ ]:
loom=scv.read('loom_afterconcat.h5ad')

In [ ]:
adata=scv.utils.merge(dcdata, loom).copy()

In [ ]:
sc.set_figure_params(6)
sc.pl.umap(adata,color=['PatientID'],ncols=2)

In [ ]:
adata=sc.read(WD+'/analyzed/'+analysis_name+'/'+analysis_name+'.annotated.'+velosubset+'.h5ad')

In [ ]:
sc.pl.umap(adata,color=['leiden'],legend_loc='on data')

In [ ]:
#catorder=['8','12','10','6','4','5','3','1']
catorder=corder=['2','5','10','7','13','8']

In [ ]:
#adata=adata[adata.obs['leiden'].isin(catorder)].copy()

In [ ]:
scv.pp.filter_and_normalize(adata)
scv.pp.moments(adata)
scv.pp.neighbors(adata)

scv.tl.recover_dynamics(adata)
scv.tl.velocity(adata,mode='dynamical')
scv.tl.velocity_graph(adata)

In [ ]:
mysubset='All'
subcat='PBMCandTIL'


In [ ]:
#subcat='TILonly'
adata.write(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.h5ad')

In [ ]:
#adata=sc.read(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.h5ad')

In [ ]:
#scv.pl.velocity(adata, ['NR4A2',  'ZNF331', 'CREM','PRDM1','LYST','PKM'], ncols=2,groupby='leiden')

In [ ]:
scv.pl.scatter(adata, 'CREM', color=['celltype3', 'velocity']) #               add_outline='ProgExh, TermExh'

In [ ]:
scv.pl.scatter(adata, 'NR4A2', color=['celltype3', 'velocity']) #    

In [ ]:
scv.tl.velocity_confidence(adata)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(adata, c=keys, cmap='coolwarm', perc=[5, 95])


In [ ]:
scv.pl.velocity_graph(adata, threshold=.2, color='leiden')

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color=['leiden','Sample type'])

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color=['leiden','Sample type'])

In [ ]:
sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,10)

In [ ]:
figdir=outpath


In [ ]:
figdir=os.path.join(outpath+'/publication/')
scv.settings.figdir = figdir
if not os.path.exists(figdir):
    os.makedirs(figdir)

In [ ]:
scv.tl.score_genes_cell_cycle(adata)
scv.pl.scatter(adata, color_gradients=['S_score', 'G2M_score'], smooth=True, perc=[5, 95],
               save='CellCycle-velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn-CCPhase.png')
s_genes, g2m_genes = scv.utils.get_phase_marker_genes(adata)
#s_genes = scv.get_df(adata[:, s_genes], 'spearmans_score', sort_values=True).index
#g2m_genes = scv.get_df(adata[:, g2m_genes], 'spearmans_score', sort_values=True).index
#kwargs = dict(frameon=False, ylabel='cell cycle genes')
#scv.pl.scatter(adata, list(s_genes[:2]) + list(g2m_genes[:3]), **kwargs)
#save='CellCycle-velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.png'

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color=['Sample type'],
                                 min_mass=3.25, 
                                 save='-velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn-SampleType.png')

In [ ]:
sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,10)

In [ ]:

scv.pl.velocity_embedding_stream(adata, basis='umap',color=['celltype3_merged'], 
                                 legend_loc='side', min_mass=3.25,
                                save='-velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn-celltype3.png')

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color=['leiden'], legend_loc='on data')

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color=['leiden_TNK'], legend_loc='on data')

In [ ]:
sc.set_figure_params()

In [ ]:
sc.pl.umap(adata,color=['XCL1','CCR7','SELL','MKI67', 'STMN1','ENTPD1','CD38','TOX'], color_map='viridis')

In [ ]:
sc.pl.umap(adata,color=['leiden'],legend_loc='on data')

In [ ]:
sc.pl.umap(adata,color=['leidenf'],legend_loc='on data')

In [ ]:
#adata.uns['iroot'] = np.flatnonzero(((adata.obs['leiden']  == '11')&(adata.obs['celltype3']  == 'naive thymus-derived CD8-positive, alpha-beta T cell')&(adata.obs['Sample type']  == 'TIL')))[0]
#adata.uns['iroot'] =np.flatnonzero(((adata.obs['root_cells']>0.8)&((adata.obs['celltype3']  == 'naive thymus-derived CD8-positive, alpha-beta T cell')|(adata.obs['celltype3_TNK']  == 'NaiCD8Tcell'))))[0]
#root_key='iroot'

In [ ]:
adatapbmc=adata[adata.obs['Sample type']=='PBMC'].copy()
adatatil=adata[adata.obs['Sample type']=='TIL'].copy()

In [ ]:
set(adata.obs['celltype3_merged'])

In [ ]:
adatanocc=adata[adata.obs['celltype3_merged']!='proliferating CD8-positive, alpha-beta T cell'].copy()

In [ ]:
scv.tl.latent_time(adatanocc,min_likelihood=0.2, min_confidence=0.9) # #,root_key='iroot'
#scv.tl.latent_time(adata)
scv.pl.scatter(adatanocc, color='latent_time', color_map='gnuplot', size=30)

In [ ]:
scv.tl.latent_time(adatapbmc,min_likelihood=0.2, min_confidence=0.9) # #,root_key='iroot'
#scv.tl.latent_time(adata)
scv.pl.scatter(adatapbmc, color='latent_time', color_map='gnuplot', size=30)

In [ ]:
scv.tl.latent_time(adatatil,min_likelihood=0.2, min_confidence=0.9) # #,root_key='iroot'
#scv.tl.latent_time(adata)
scv.pl.scatter(adatatil, color='latent_time', color_map='gnuplot', size=30)

In [ ]:
scv.tl.latent_time(adata,min_likelihood=0.2, min_confidence=0.9) # #,root_key='iroot'
#scv.tl.latent_time(adata)
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot', size=30)

In [ ]:
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot', size=30)

In [ ]:
#sc.pl.umap(adata[adata.obs['root_cells']>0.5], color=['leiden','celltype3'])

In [ ]:
sc.pl.umap(adata, color=['root_cells','end_points'])

In [ ]:
sc.pl.umap(adatanocc, color=['root_cells','end_points'])

In [ ]:
adata.write(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.h5ad')



In [ ]:
adata=sc.read(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.h5ad')

In [ ]:
adata.obs['RCat']=adata.obs['Response3'].copy()
adata.obs['RCat']=list(adata.obs['RCat'].replace('NR', 'NR_nadj'))
adata.obs['RCat']=list(adata.obs['RCat'].replace('PD', 'NR_adj'))

In [ ]:
### Remove two uncertain patients
ddata=adata[adata.obs['PatientID']!='P38'].copy()
ddata=adata[adata.obs['PatientID']!='P12'].copy()

In [ ]:
ddata.obs['PatientID']=ddata.obs['PatientID'].astype(str)

In [ ]:
ddata.obs['R_clus']=ddata.obs['RCat'].astype(str)+'-'+[x.replace("-", "_") for x in ddata.obs['celltype3_merged'].astype(str)]
ddata.obs['Pat_clus']=ddata.obs['PatientID'].astype(str)+'-'+[x.replace("-", "_") for x in ddata.obs['celltype3_merged'].astype(str)]
dfPat = ddata.obs.groupby('Pat_clus')['velocity_length', 'velocity_confidence','latent_time'].mean().T

In [ ]:
dfPat

In [ ]:
Resp=[ddata[ddata.obs['PatientID']==x.split('-')[0]].obs['RCat'][0] for x in list(dfPat.columns)]
PatientID=[x.split('-')[0]  for x in list(dfPat.columns)]
ClusterID=[x.split('-')[1]  for x in list(dfPat.columns)]

In [ ]:
mydf=pd.DataFrame([list(dfPat.loc['velocity_length',:]),Resp,PatientID,ClusterID]).transpose()
mydf.index=dfPat.columns
mydf.columns=['velocity_length','Response','PatientID','Cluster']

mydfl=pd.DataFrame([list(dfPat.loc['latent_time',:]),Resp,PatientID,ClusterID]).transpose()
mydfl.index=dfPat.columns
mydfl.columns=['latent_time','Response','PatientID','Cluster']

In [ ]:
mydf.groupby('PatientID', as_index=False)['velocity_length']

In [ ]:
color_dict = {'R': 'coral', 'TF': 'firebrick', 'NR_nadj': 'lightskyblue','NR_adj': 'royalblue'}

#### Velocity test first

In [ ]:
#corder=['8','6','1','21']
mydf['velocity_length']=mydf['velocity_length'].astype('float')
velMean=mydf.groupby('PatientID')['velocity_length'].mean()
patid=[ddata[ddata.obs['PatientID']==x.split('-')[0]].obs['RCat'][0]  for x in list(velMean.index)]
totest=pd.DataFrame([list(velMean),patid]).transpose()
totest.columns=['Val','Cat']

In [ ]:
ss.mannwhitneyu(list(totest.loc[totest['Cat']=='R',:]['Val']),list(totest.loc[totest['Cat']=='NR_nadj',:]['Val']))

In [ ]:
ss.mannwhitneyu(list(totest.loc[totest['Cat']=='TF',:]['Val']),list(totest.loc[totest['Cat']=='NR_adj',:]['Val']))

In [ ]:
set(mydf['Response'])

In [ ]:
totestk=totest.copy()

In [ ]:
fig=sns.boxplot(y='Val',x='Cat',data=totestk,palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
fig=sns.swarmplot(x='Cat',y='Val',data=totestk,color='black',order=['R','TF','NR_nadj','NR_adj'])
fig.figure.savefig(figdir+'Velocitylen-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.pdf') 

In [ ]:
totest=list(itertools.combinations(['R','TF','NR_nadj','NR_adj'], 2))
pwilc={}
pt={}
for pairs in totest:
    pwilc[pairs[0]+'-'+pairs[1]]=stats.mannwhitneyu(list(totestk.loc[totestk['Cat']==pairs[0],:]['Val']), 
                           list(totestk.loc[totestk['Cat']==pairs[1],:]['Val']))[1]
    pt[pairs[0]+'-'+pairs[1]]=stats.ttest_ind(list(totestk.loc[totestk['Cat']==pairs[0],:]['Val']), 
                           list(totestk.loc[totestk['Cat']==pairs[1],:]['Val']))[1]

myps=pd.DataFrame([pwilc,pt]).transpose()
myps.columns=['MannWhitney','T-test']

myps.to_csv(figdir+'Pval-Velocitylen-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.pdf',sep='\t')

In [ ]:
myps

In [ ]:
corder=list(set(mydf['Cluster']))

In [ ]:
for i in corder:
    sns.boxplot(y='velocity_length',x='Response',data=mydf.loc[mydf['Cluster']==i,:],
                palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
    sns.swarmplot(x='Response',y='velocity_length',data=mydf.loc[mydf['Cluster']==i,:],
                  color='black',order=['R','TF','NR_nadj','NR_adj'])
    print('Cluster '+i)
    plt.figure()

#### Latent time next

In [ ]:
mydfl

In [ ]:
#corder=['8','6','1','21']
mydfl['latent_time']=mydfl['latent_time'].astype('float')
velMean=mydfl.groupby('PatientID')['latent_time'].mean()
patid=[ddata[ddata.obs['PatientID']==x.split('-')[0]].obs['RCat'][0]  for x in list(velMean.index)]
totest=pd.DataFrame([list(velMean),patid]).transpose()
totest.columns=['Val','Cat']

ss.mannwhitneyu(list(totest.loc[totest['Cat']=='R',:]['Val']),list(totest.loc[totest['Cat']=='NR_nadj',:]['Val']))



In [ ]:
ss.mannwhitneyu(list(totest.loc[totest['Cat']=='TF',:]['Val']),list(totest.loc[totest['Cat']=='NR_adj',:]['Val']))

In [ ]:
totestk=totest.copy()

In [ ]:
fig=sns.boxplot(y='Val',x='Cat',data=totestk,palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
fig=sns.swarmplot(x='Cat',y='Val',data=totestk,color='black',order=['R','TF','NR_nadj','NR_adj'])
fig.figure.savefig(figdir+'LatentTime-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.pdf') 

In [ ]:
corder=list(set(mydf['Cluster']))

for i in corder:
    sns.boxplot(y='latent_time',x='Response',data=mydfl.loc[mydf['Cluster']==i,:],
                palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
    sns.swarmplot(x='Response',y='latent_time',data=mydfl.loc[mydfl['Cluster']==i,:],
                  color='black',order=['R','TF','NR_nadj','NR_adj'])
    print('Cluster '+i)
    plt.figure()

### Kinetic rate paramters

The rates of RNA transcription, splicing and degradation are estimated without the need of any experimental data.

They can be useful to better understand the cell identity and phenotypic heterogeneity.

In [ ]:
#adata.var['velocity_genes']

In [ ]:
df = adata.var
df = df[(df['fit_likelihood'] > .1) & df['velocity_genes'] == True]

kwargs = dict(xscale='log', fontsize=16)
with scv.GridSpec(ncols=3) as pl:
    pl.hist(df['fit_alpha'], xlabel='transcription rate', **kwargs)
    pl.hist(df['fit_beta'] * df['fit_scaling'], xlabel='splicing rate', xticks=[.1, .4, 1], **kwargs)    
    pl.hist(df['fit_gamma'], xlabel='degradation rate', xticks=[.1, .4, 1], **kwargs)
    
scv.get_df(adata, 'fit*', dropna=True).head()

The estimated gene-specific parameters comprise rates of transription (`fit_alpha`), splicing (`fit_beta`), degradation (`fit_gamma`), switching time point (`fit_t_`), a scaling parameter to adjust for under-represented unspliced reads (`fit_scaling`), standard deviation of unspliced and spliced reads (`fit_std_u`, `fit_std_s`), the gene likelihood (`fit_likelihood`), inferred steady-state levels (`fit_steady_u`, `fit_steady_s`) with their corresponding p-values (`fit_pval_steady_u`, `fit_pval_steady_s`), the overall model variance (`fit_variance`), and a scaling factor to align the gene-wise latent times to a universal, gene-shared latent time (`fit_alignment_scaling`).

### Latent time

The dynamical model recovers the latent time of the underlying cellular processes. This latent time represents the cell’s internal clock and approximates the real time experienced by cells as they differentiate, based only on its transcriptional dynamics. 

In [ ]:
#scv.tl.terminal_states(adata)
#scv.pl.scatter(adata, color=[ 'root_cells', 'end_points'])
#tl.latent_time(data[, vkey, min_likelihood, . . . ]) C
#scvelo.tl.latent_time(data, vkey=’velocity’, min_likelihood=0.1, min_confidence=0.75,
#min_corr_diffusion=None, weight_diffusi

In [ ]:
scv.tl.rank_dynamical_genes(adata, groupby='leiden')


In [ ]:
df = scv.get_df(adata, 'rank_dynamical_genes/names')
df.head(10)
#r

In [ ]:
scv.tl.rank_dynamical_genes(adata, groupby='celltype3_merged')


In [ ]:
df = scv.get_df(adata, 'rank_dynamical_genes/names')
df.head(10)
#r

In [ ]:
df.head(50).to_csv(figdir+'/TopDynamicGenesPerCluster_'+velosubset+'-'+mysubset+'_'+subcat+'_loom-dyn_Top50Ranked.tsv',sep='\t')

In [ ]:

#for cluster in list(df.columns):
#    scv.pl.scatter(adata, df[cluster][:5], ylabel='celltype3_merged', frameon=False)


In [ ]:
ofinterest=set([item for sublist in df.head(25).values.tolist() for item in sublist])



In [ ]:
velogenes=adata.var.loc[(adata.var['velocity_genes']==True)&(adata.var['fit_likelihood']>.1),:]

In [ ]:
velogenes.sort_values('fit_likelihood', ascending=False)

In [ ]:
set(list(velogenes.index))

In [ ]:
top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index#[:300]


In [ ]:
invelo=[]
for x in list(top_genes):
    invelo=invelo+list(set([x]).intersection(set(velogenes.index)))

In [ ]:
scv.pl.heatmap(adata, var_names=invelo, sortby='latent_time', 
               col_color='celltype3_merged', n_convolve=30,
               font_scale=0.5,  figsize=(8, 6))

In [ ]:
scv.pl.heatmap(adata, var_names=invelo[0:50], 
               sortby='latent_time', col_color='celltype3_merged', n_convolve=30,
               font_scale=0.5,  figsize=(8, 6))

In [ ]:
scv.pl.heatmap(adata, var_names=invelo[50:100], 
               sortby='latent_time', col_color='celltype3_merged', n_convolve=30,
               font_scale=0.5,  figsize=(8, 6))

In [ ]:
if (len(invelo)>100):
    scv.pl.heatmap(adata, var_names=invelo[100:len(invelo)], 
               sortby='latent_time', col_color='celltype3_merged', n_convolve=30,
               font_scale=0.5,  figsize=(8, 6))

In [ ]:
velogenes.sort_values('fit_likelihood', ascending=False).to_csv(figdir+'/VelocityInfoPerGene_'+velosubset+'-'+mysubset+'_'+subcat+'_loom-dyn_allVeloGenesRanked.tsv',sep='\t')

In [ ]:
tkey='latent_time'
var_names=invelo
var_names = [name for name in var_names if name in adata.var_names]
xkey='Ms'
time = adata.obs[tkey].values
time = time[np.isfinite(time)]

df = pd.DataFrame(adata[:, var_names].layers[xkey][np.argsort(time)], columns=var_names)

In [ ]:
list(adata.obs.columns)

In [ ]:
max_sort = np.argsort(np.argmax(df.values, axis=0))
df = pd.DataFrame(df.values[:, max_sort], columns=df.columns[max_sort])
df.index=pd.DataFrame([adata.obs['celltype3_merged'][np.argsort(time)],
                       adata.obs['leiden'][np.argsort(time)],adata.obs['latent_time'][np.argsort(time)]]).transpose().index
df=pd.concat([df,pd.DataFrame([adata.obs['celltype3_merged'][np.argsort(time)],
                               adata.obs['leiden'][np.argsort(time)],adata.obs['latent_time'][np.argsort(time)]]).transpose()], axis=1)

In [ ]:
def getMeans(adata,myvar):
    gene_ids = adata.raw.var.index.values
    clusters = adata.obs[myvar].cat.categories
    obs = adata.raw[:,gene_ids].X.toarray()
    obs = pd.DataFrame(obs,columns=gene_ids,index=adata.obs[myvar])
    average_obs = obs.groupby(level=0).mean()
    obs_bool = obs.astype(bool)
    fraction_obs = obs_bool.groupby(level=0).sum()/obs_bool.groupby(level=0).count()
    return(average_obs,fraction_obs)


In [ ]:
avg,fct=getMeans(adata,'leiden')

In [ ]:
avg.loc[:,ofinterest]

In [ ]:
scv.settings.set_figure_params('scvelo')

plt.figure(figsize=(4,3))
mydat=ddata.obs.loc[:,['RCat','latent_time']]
fig=sns.violinplot(x='RCat',y='latent_time',data=mydat, palette=color_dict, order=['R','TF','NR_nadj','NR_adj'])
fig.figure.savefig(figdir+'LatentTime-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response-violins.pdf') 

In [ ]:
lt={}
for resp in  list(color_dict.keys()):
    lt[resp]=ddata[ddata.obs['RCat']==resp].obs['latent_time']
mydat=ddata.obs.loc[:,['RCat','latent_time']]

plt.figure(figsize=(4,3))
myc=list(color_dict.values())
i=0
# Iterate through the five airlines
for resp in list(color_dict.keys()):
    subset = lt[resp]
    sns.distplot(subset, hist = False, kde = True,
                 kde_kws = {'linewidth': 3},
                 label = resp,color=myc[i])
    i=i+1

# Plot formatting
plt.legend(prop={'size': 10}, title = 'Responders')
plt.title('Density Plot across Responses')
plt.xlabel('Latent time')
plt.ylabel('Density')
#plt.savefig(outpath+'/LatentTime_RvsNR_CD8TNK_CCtoExh_TILonly.pdf')
plt.savefig(figdir+'LatentTime-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response-desity.pdf')

In [ ]:
### check avg. per patient as well
r_nr=ss.mannwhitneyu(x=mydat.loc[mydat['RCat']=='R','latent_time'], y=mydat.loc[mydat['RCat']=='NR_nadj','latent_time'])[1]
### check avg. per patient as well
r_tf=ss.mannwhitneyu(x=mydat.loc[mydat['RCat']=='R','latent_time'], y=mydat.loc[mydat['RCat']=='TF','latent_time'])[1]
### check avg. per patient as well
r_pd=ss.mannwhitneyu(x=mydat.loc[mydat['RCat']=='R','latent_time'], y=mydat.loc[mydat['RCat']=='NR_adj','latent_time'])[1]
### check avg. per patient as well
tf_pd=ss.mannwhitneyu(x=mydat.loc[mydat['RCat']=='TF','latent_time'], y=mydat.loc[mydat['RCat']=='NR_adj','latent_time'])[1]
### check avg. per patient as well
tf_nr=ss.mannwhitneyu(x=mydat.loc[mydat['RCat']=='TF','latent_time'], y=mydat.loc[mydat['RCat']=='NR_nadj','latent_time'])[1]
### check avg. per patient as well
pd_nr=ss.mannwhitneyu(x=mydat.loc[mydat['RCat']=='NR_adj','latent_time'], y=mydat.loc[mydat['RCat']=='NR_nadj','latent_time'])[1]
myp=pd.Series([r_nr,r_tf,r_pd,tf_pd,tf_nr,pd_nr])
myp.index=['R-NR_nadj','R-TF','R-NR_adj','TF-NR_adj','TF-NR_nadj','NR_adj-NR_nadj']
myp

In [ ]:
plt.figure(figsize=(4,3))
(-1*np.log10((myp+1.163760e-80).sort_values())).plot.bar()
plt.ylabel('-log10(p)')
plt.savefig(figdir+'LatentTime-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response-violins-pvals.pdf') 

In [ ]:
latcat=adata.obs['leiden'].copy()
latcat=latcat.cat.set_categories(['early','mid','late'])

In [ ]:
latcat[adata.obs['latent_time']<=0.25]='early'
latcat[adata.obs['latent_time']>0.5]='late'
latcat[(adata.obs['latent_time']>0.25)&(adata.obs['latent_time']<=0.5)]='mid'
adata.obs['latcat']=list(latcat)

sc.pl.umap(adata,color=['latcat'],legend_loc='on data')

In [ ]:
! jupyter nbconvert --to html scvelo-monomacro.ipynb